# 📍Example 5: P-Median Problem using Gurobi

This example demonstrates solving the **P-Median Problem** using **Gurobi** in Python. The objective is to locate exactly **p facilities** among potential sites to **minimize the total weighted distance** between demand points and their assigned facilities.

P-Median is widely used in **location analysis** for services like warehouses, hospitals, or distribution centers where minimizing travel or service distance is critical.

---

## 🔢 Mathematical Formulation

We want to:

**Minimize:**

$$
\sum_{i \in I} \sum_{j \in J} d_{ij} \cdot y_{ij}
$$

**Subject to:**

**Demand Assignment:**

$$
\sum_{j \in J} y_{ij} = 1, \quad \forall i \in I
$$

**Assignment only to open facilities:**

$$
y_{ij} \leq x_j, \quad \forall i \in I, j \in J
$$

**Facility count constraint:**

$$
\sum_{j \in J} x_j = p
$$

**Binary variables:**

$$
x_j \in \{0,1\}, \quad \forall j \in J
$$

$$
y_{ij} \in \{0,1\}, \quad \forall i \in I, j \in J
$$

---

**Where:**

- $x_j = 1$ if a facility is opened at site $j$, else 0  
- $y_{ij} = 1$ if demand point $i$ is assigned to facility $j$, else 0  
- $d_{ij}$ = distance from demand point $i$ to facility $j$  
- $p$ = total number of facilities to locate  
- $I$ = set of demand points, $J$ = set of potential facility sites  

---

## 📊 Problem Instance

- Demand Points: D1, D2, D3, D4  
- Potential Facilities: F1, F2, F3  
- Number of Facilities to open ($p$): 2

**Distance Matrix:**

| Demand/Facility | F1 | F2 | F3 |
|-----------------|----|----|----|
| **D1**          | 2  | 4  | 5  |
| **D2**          | 3  | 2  | 6  |
| **D3**          | 4  | 3  | 2  |
| **D4**          | 5  | 6  | 3  |

---

🎯 The goal is to minimize the total distance between demand points and their assigned facilities while opening exactly 2 facilities.


In [ ]:
import gurobipy as gp
from gurobipy import GRB

# Define the data
I = ['D1', 'D2', 'D3', 'D4']
J = ['F1', 'F2', 'F3']
d = {
    ('D1', 'F1'): 2, ('D1', 'F2'): 4, ('D1', 'F3'): 5,
    ('D2', 'F1'): 3, ('D2', 'F2'): 2, ('D2', 'F3'): 6,
    ('D3', 'F1'): 4, ('D3', 'F2'): 3, ('D3', 'F3'): 2,
    ('D4', 'F1'): 5, ('D4', 'F2'): 6, ('D4', 'F3'): 3
}
p = 2

# Create a new model
model = gp.Model("p-median")

# Create variables
x = model.addVars(J, vtype=GRB.BINARY, name="x")
y = model.addVars(I, J, vtype=GRB.BINARY, name="y")

# Set objective to minimize total assignment distance
model.setObjective(gp.quicksum(d[i, j] * y[i, j] for i in I for j in J), GRB.MINIMIZE)

# Demand assignment constraints: each demand assigned to exactly one facility
model.addConstrs((gp.quicksum(y[i, j] for j in J) == 1 for i in I), name="Assign")

# Assignments only allowed to open facilities
model.addConstrs((y[i, j] <= x[j] for i in I for j in J), name="Open")

# Exactly p facilities open
model.addConstr(gp.quicksum(x[j] for j in J) == p, name="FacilityCount")

# Optimize model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal solution found:")
    for j in J:
        if x[j].x > 0.5:
            print(f"Facility located at {j}")
    for i in I:
        for j in J:
            if y[i, j].x > 0.5:
                print(f"Demand point {i} is assigned to facility {j}")
else:
    print("No optimal solution found")


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 17 rows, 15 columns and 39 nonzeros
Model fingerprint: 0x0ec5554b
Variable types: 0 continuous, 15 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 19.0000000
Presolve removed 17 rows and 15 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 2: 10 19 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%
Optimal solution found:
Facility located at F1

## Visualization

This simple graph helps you **see which demand points are assigned to which open facilities** in the P-Median solution.

### 📌 Legend:
- **Blue circles** = Demand points (customers).
- **Red circles** = Facilities that were selected (opened).
- **Gray circles** = Facilities that were not opened.
- **Gray lines** = Show which facility each demand is assigned to.

### 📊 How to Read the Graph:
- Each **line** goes from a **blue node (demand)** to a **red node (facility)**.
- Only **opened facilities** (in red) will have incoming lines.
- Facilities with no red color were **not chosen** by the optimization.

